In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from utils_fr import *
from inception_blocks import *

In [4]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
print("Total params: ", FRmodel.count_params())

Total params:  3743280


### THE TRIPLET LOSS

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$
Here, we are using the notation "$[z]_+$" to denote $max(z,0)$.

In [6]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum((anchor - positive)**2, axis=-1)
    
    neg_dist = tf.reduce_sum((anchor - negative)**2, axis=-1)
    
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    
    return loss

In [7]:
import tensorflow.compat.v1 as tf1

with tf1.Session() as sess:
    tf1.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf1.random_normal([3, 128], mean=6, stddev=0.1, seed=1),
              tf1.random_normal([3, 128], mean=1, stddev=1, seed=1),
              tf1.random_normal([3, 128], mean=3, stddev=4, seed=1))

    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))

loss = 528.1426


In [8]:
FRmodel.compile(optimizer="adam", loss=triplet_loss, metrics=["accuracy"])
load_weights_from_FaceNet(FRmodel)

In [11]:
db = {}
db["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
db["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)
db["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
db["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
db["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
db["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
db["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
db["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
db["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
db["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
db["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
db["tian"] = img_to_encoding("images/tian.jpg", FRmodel)

In [17]:
def verify(img_path, identity, db, model):
    threshold = 0.7
    encoding = img_to_encoding(img_path, model)
    dist = np.linalg.norm(db[identity] - encoding)
    
    if dist < threshold:
        print("It's ", identity, ", welcome home!")
        door_open = True
    else:
        print("It's not ", identity, ", please go away.")
        door_open = False
        
    return dist, door_open

In [18]:
verify("images/camera_0.jpg", "younes", db, FRmodel)

It's  younes , welcome home!


(0.6671397, True)

In [19]:
verify("images/camera_2.jpg", "kian", db, FRmodel)

It's not  kian , please go away.


(0.8586888, False)

### Face Recognition

In [22]:
def who_is_it(img_path, db, model):
    encoding = img_to_encoding(img_path, model)
    
    min_dist = 100
    
    for (name, db_enc) in db.items():
        dist = np.linalg.norm(encoding - db_enc)
        
        if dist < min_dist:
            min_dist = dist
            identity = name
            
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print("It's ", identity, ", the distace is ", min_dist)
        
    return min_dist, identity

In [23]:
who_is_it("images/camera_0.jpg", db, FRmodel)

It's  younes , the distace is  0.6671397


(0.6671397, 'younes')